In [1]:
#!pip install sympy

In [2]:
from pyomo.environ import *
import numpy as np
from math import pi
from Sympy_OPF_LM_class import SympyACOPFModel
from Sympy_OPF_LM_class import PrintQHDACOPFResults
from qhdopt import QHD

In [3]:
# 初始化（用默认 3-bus 数据）
model = SympyACOPFModel()   # 打开 proximal 选项，后面会用到

In [4]:
#Lagrange Loop

# 3. 构建 h_func
h_func = model.build_h_func()   # 默认 reference bus = 第一个母线
model.reset_lambdas(0.0)

alpha = 1e-3   # 对偶步长尽量小一点
gamma = 0.1    # proximal 权重，可以从 0.01~1 慢慢调

x_k = None     # 初始没有 proximal 中心

for round in range(10):

    # ---- (0) 设置 proximal 中心 ----
    # 第一轮没有历史解，可以关掉 proximal；之后就用上一次解做中心
    if x_k is None:
        model.set_proximal(x_center=None, gamma=0.0)
    else:
        model.set_proximal(x_center=x_k, gamma=gamma)

    # ---- (1) 用当前 λ 和 proximal，构造 L(x, λ) ----
    # 拿到 Lagrangian + 变量列表 + bound 列表
    Lag, variable_list, Var_bound_list = model.get_Lagrangian()

    # ---- (2) QHD backend 求解: min_{box} L(x, λ^k, proximal) ----
    qhd_model = QHD.SymPy(Lag, variable_list, Var_bound_list)
    #model.qutip_setup(4, embedding_scheme='onehot', time_discretization=40, post_processing_method="TNC")
    qhd_model.simbi_setup(resolution=4, agents= 1024, max_steps= 500,embedding_scheme="unary", post_processing_method='TNC', verbose = False)
    response = qhd_model.optimize(verbose=0)

    #print("round: ", round, " solution: ", response.refined_minimizer, len(response.refined_minimizer), len(variable_list))
    PrintQHDACOPFResults(model, response.refined_minimizer)

    # ---- (3) primal 更新：记住解作为下一轮 proximal 中心 ----
    x_k = np.asarray(response.refined_minimizer)

    # ---- (4) 对偶更新：λ^{k+1} = λ^k + α h(x^{k+1}) ----
    # 现在重新 build 拉格朗日，就是用新的 λ 了
    lambda_new, h_x = model.update_lambda(x_k, alpha, h_func)
    model.reset_lambdas(lambda_new)

    #print("round: ", round, " lambda_new: ", lambda_new, "round", round, "||h||=", np.linalg.norm(h_x))

    # ---- (5) 可行性检查（可选） ----
    _, Constraint_check_marker = model.check_constraints(response.refined_minimizer)
    print("round: ", round, " concstraint check ", Constraint_check_marker)

c:\Users\zzhao3\AppData\Local\anaconda3\envs\QHD_SB_cyipopt\lib\site-packages\qhdopt\qhd.py:47: RuntimeWarning: The number of function free symbols does not match the number of syms.
  warnings.warn("The number of function free symbols does not match the number of syms.",


backend time consumption: 0.4551734924316406
BusID	VR	VI	Pg	Qg	l	Pl	Ql	Qshunt

1	0.5500	0.5500	0.0000	7.0000	1.0000	0.0000	0.0000	0.0000
2	0.5500	0.5500	0.0000	7.0000	1.0000	0.0000	0.0000	0.0000
3	0.5500	0.5500	0.0000	0.0000	1.0000	0.3000	0.1000	0.0000


TOTAL			0.0000	14.0000		0.3000	0.1000



Busi	Busk	Pik	Pki	Qik	Qki
1	2	1.5000	0.0000	1.5000	0.0000
1	3	1.5000	0.0000	1.5000	0.0000
2	3	1.5000	0.0000	1.5000	0.0000


Total Ploss: 4.5000
Total Qloss: 4.5000
Total Load Supplied: -1500.0000%
round:  0  concstraint check  False
backend time consumption: 0.21453070640563965
BusID	VR	VI	Pg	Qg	l	Pl	Ql	Qshunt

1	1.1000	1.1000	0.0000	6.9303	1.0000	0.0000	0.0000	0.0000
2	-1.1000	-1.1000	0.0000	6.9286	1.0000	0.0000	0.0000	0.0000
3	0.5917	0.5112	0.0000	0.0000	1.0000	0.3000	0.1000	0.0000


TOTAL			0.0000	13.8589		0.3000	0.1000



Busi	Busk	Pik	Pki	Qik	Qki
1	2	1.5547	25.2873	1.5547	75.7302
1	3	1.5547	-1.1993	1.5547	-2.9566
2	3	1.5547	3.5718	1.5547	9.3326


Total Ploss: 32.3240
Total Qloss: 86.7704
To